### Main program on estimating error function for Off-Policy-Evaluation

In [1]:
# !pip install Box2D
# !pip install 'gym[all]'
# !pip install pyyaml
import gym
import numpy as np
import matplotlib.pyplot as plt
from collections import deque, defaultdict

import time
import sys
from tqdm import tqdm
import yaml
import re

from dqnetwork import DQNetwork
from agent import Agent

import torch
import torch.nn as nn
import torch.nn.functional as F

from os import listdir, getcwd
from os.path import isabs, join

In [2]:
## Load the environment
env_id = 'CartPole-v0'
env = gym.make(env_id)

## Step 1. Load Policies

In [3]:
### get all model policies - "../model/policies/behavior_policy_x.pth"
def get_policies(path, config):
    """
    Loads all policies in a given directory into an agent
    @Param:
    1. Path - model path.
    2. config - path for configuration of corresponding model
    @Return:
    - policies: list of agent
    """
    cwd = getcwd() #get current working directory
    index = lambda index : int(re.findall(r'[0-9]+', index)[-1]) #get policy number
        
    #Get paths for config and policy.
    policy_files = {index(f): join(path, f) for f in listdir(join(cwd, path)) if isabs(join(cwd, f))}
    yaml_files   = [join(cwd, join(config, f)) for f in listdir(join(cwd, config)) if isabs(join(cwd, f))]

    policies = [] #loads an agent policy
    
    for config_file in yaml_files:
        with open(r''+ config_file) as file:
            config_data = yaml.load(file, Loader=yaml.FullLoader) #get output as dict
            info = config_data[0] #gets model size info
            num = index(config_file) #get file number
            agent = Agent(fc1=info['fc1'], fc2=info['fc2'], path=policy_files[num])
            policies.append( agent ) #add agent
    return policies

In [4]:
agents = get_policies("../model/policies", "../model/config")

Model loaded into local and target networks!
Model loaded into local and target networks!
Model loaded into local and target networks!


## Step 2 - generate policy matrix
<p> A policy matrix is a dictionary of dimensions (K, K - 1), where K is the total number of behavior policies </p>

In [5]:
def policy_matrix(agents):
    """Generates policy matrix (dictionary) of shape (n, n-1) for x agents"""
    matrix = {}
    for i, evaluation in enumerate(agents):
        matrix[evaluation] = []
        for j, behavior in enumerate(agents):
            if(i != j):
                matrix[evaluation].append(behavior)
    return matrix        

In [6]:
#each key indicates evaluation policy, and corresponding values indicate behavior policies
policy_dict = policy_matrix(agents)

In [7]:
def get_behavior_policies(evaluation_policy):
    """Generates respective behvaior policies for a particular evaluation policy"""
    return policy_dict[evaluation_policy] if evaluation_policy in policy_dict.keys() else None

## Step 3. Get Horizons

$$ \xi_k =  \prod_{t=1}^{H} \frac{ \pi_e({a_t}^k | {s_t}^k) }{ \pi_k({a_t}^k | {s_t}^k) } $$

In [8]:
def horizon(evaluation_agent, behavior_agent):
    """
    > Calculates ratio between an evaluation and a behavior agent for infinite horizon
    > Calculates Return value (total reward) for behavior_agent
    NOTE: behavior_agent (π_k) generates all states and action.
    @Param:
    1. evaluation_agent - agent class object representing evaluation policy.
    2. behavior_agent - agent class object representing behavior policy.
    @Returns:
    - ratio : value ratio b/w evaluation and behavior agent.
    - total_reward : return of behavior_agent.
    """
    ratio = 1
    total_reward = 0
    state = env.reset() #reset
    while True:
        action, prob_behv = behavior_agent.get_action(state, eps=0) #generate best action and prob for behavior.
        _, prob_eval = evaluation_agent.get_action(state, eps=0) #generate max probability for evaluation policy.
        
        ratio *= float(prob_eval/prob_behv) #compute ratio
        
        next_state, reward, done, info = env.step(action) #transition
        
        total_reward += reward #update reward
        state = next_state #update state
        
        if(done): #stopping condition
            break
    
    return ratio, total_reward

## Step 4. Get Trajectories

$$ \sigma(\pi_e, \pi_k) = \sum\limits_{i=1}^N {R_k}^i \times {\xi_k}^i  $$

In [9]:
def get_trajectories(evaluation_agent, behavior_agent, N, type):
    """
    See formula above.
    Calculates the sum of value for a behavior policy and corresponding evaluation policy
    based on N trajectories.
    
    > Computes the sum of ratio b/w evaluation and behavior agent, 𝜉
    > Compute sigma that represents inner sum for 1 behavior policy and corresponding evaluation policy
    
    @Param:
    1. evaluation_agent - (Agent) Evaluation Policy
    2. behavior_agent - (Agent) Behavior Policy
    3. N - (int) number of trajectories
    4. type - (int) 0/1 representing x_1 (0) and x_2 (1) for matrix multiplication later.
    @Return 
    - sigma = (float) returns inner sum, sigma (see formula for more details).
    """
    sigma = 0
    for _ in range(N):
        reward, xi = horizon(evaluation_agent, behavior_agent)
        sigma += reward * xi if(type == 0) else reward #X_2 doesn't use xi because it's distributed later.
    return sigma

## Step 5. Calculate function value

$$ f(\hat{\pi}) = \frac{1}{K N} \sum\limits_{k=1}^K \sigma(\pi_e, \pi_k) $$

In [10]:
def function(behavior_agents, evaluation_agent, N, type):
    """
    Computes f(π) using formula shown above.
    @Param:
    1. behavior_agent: (list) list of Agent class objects representing set of evaluation for given π_e.
    2. evaluation_agents: π_e (Agent) behavior policy
    3. N - (int) number of trajectories (used in calculation of sigma in `get_trajectories`)
    4. type: (int) 0/1 for calculation of X_1/X_2 with regards sum of products w.r.t constants, c and d.
    @Return
    - value: (float) value of function f(π) using formula above.
    """
    value = 0
    K = len(behavior_agents)
    for agent in behavior_agents:
        sigma = get_trajectories(evaluation_agent, agent, N, type)
        value += sigma
    
    return float(value / (K * N))

## Step 6. True Value function for an evaluation agent

$$ V(\pi_e) = \frac{1}{N} \sum\limits_{i=1}^N {R_e}^i $$

In [11]:
def Value(policy_dict, N):
    """
    Calculates the expected return for all evaluation agents
    @Param: 
    1. policy_dict - (dict[list]) policy matrix of shape (n, n-1) for n agents.
    2. N - (int) number of trajectories to run value estimation for.
    NOTE: N should equal with estimation value function parameter N.
    @Return:
    - values - (nd.array) Vector of values of evaluation policies using formula shown above. 
    """
    evaluation_agents = list(policy_dict.keys()) #generate n eval agents
    values = [0]*len(evaluation_agents) #vector of values of evaluation policies
    
    for i, agent in enumerate(evaluation_agents):
        value = [] #stores N Return for agent with policy π_e.
        for n in range(N):
            total_reward = 0
            state = env.reset() #reset
            while True:
                action, prob = agent.get_action(state, eps=0)
                next_state, reward, done, info = env.step(action)
                total_reward += reward
                state = next_state
                if(done): break
                    
            #append to value
            value.append(total_reward)
        #compute mean return and store in vector of values.
        expected_val = np.mean(value)
        values[i] = expected_val
    
    return np.array([values]).T #dim = number of agents.

## Step 7. Tying it al together!

In [12]:
#retrieve an evaluation policy with agent of index i.
get_eval_agent = lambda i: list(policy_dict.keys())[i - 1]

In [13]:
def main(policy_dict, N):
    """
    Main function for generating X_1 and X_2 of shapes k each, where k = number of base policies
    @Param:
    1. policy_dict - (dict[list]) policy matrix of shape (n, n-1) for n agents.
    2. N - (int) number of trajectories to run value estimation for.
    @Return:
    - X: (nd.array) concatenation of X_1 and X_2
    """
    X_1, X_2 = [], [] #store X_1 and X_2 for k policies
    K = len(policy_dict.keys())
    
    for i in range(1, K + 1):
        ### compute X_1 with evaluation policy, π_e = π_i
        evaluation_agent = get_eval_agent(i)
        ### generate set of behavior policies for π_e = π_1, i.e. π_k = {[π_j] for j ≠ i}
        behavior_agents = get_behavior_policies(evaluation_agent)
        x1 = function(behavior_agents, evaluation_agent, N, 0) #compute x1
        x2 = function(behavior_agents, evaluation_agent, N, 1) #compute x2
        ### store values
        X_1.append(x1)
        X_2.append(x2)
        
    #typecast to nd.array
    X_1 = np.array([X_1])
    X_2 = np.array([X_2])
    X = np.hstack((X_1.T, X_2.T)) #concat
    
    #Test dimensions
    assert(X.shape == (K, 2))
    return X

In [14]:
### calculate X with 50 trajectories
X = main(policy_dict, 50)
### generate true value estimate
true_values = Value(policy_dict, 50)

In [15]:
c, d = 2e-21, 0
Y_intercept = np.array([[c, d]]).T

output = np.dot(X, Y_intercept)
error = np.sum(np.square(output - true_values))
print("Error = ", error)

Error =  93028.98539208976
